# Agentes para Devs — Tutorial com LangChain + Gemini

Este notebook cobre: preparação do ambiente em Python/VSCode, visão da arquitetura de agentes (runtimes, ferramentas, memória), introdução à LCEL (LangChain Expression Language) e um "Hello, Agent!" com Gemini.

## Sumário
1. Setup do ambiente (Python, virtualenv e VSCode)
2. Visão geral da arquitetura de um agente
3. Introdução ao LangChain e à LCEL
4. Hello, Agent! (LLM + Parser + Prompt)
5. Extensão: Memória de conversa
6. Extensão: Ferramentas (Tools) com Gemini
7. Próximos passos

## 1. Setup do ambiente (Python, virtualenv e VSCode)

1. **Instale Python 3.10+**: verifique com `python --version`.
2. **Crie um virtualenv** no Windows:
   - `python -m venv .venv`
   - Ative: `\.venv\Scripts\activate`

   Em macOS/Linux:
   - `python3 -m venv .venv`
   - Ative: `source .venv/bin/activate`

3. **VSCode**:
   - Instale a extensão "Jupyter" (Microsoft).
   - Abra este notebook e selecione o kernel do seu virtualenv.

4. **Chave de API do Gemini**:
   - Obtenha uma chave em: https://ai.google.dev/
   - Defina `GOOGLE_API_KEY` no ambiente ou crie `.env` com `GOOGLE_API_KEY=<sua_chave>`.

5. **No terminal**:
   -  Instale todas as dependências necessárias
   - `pip install -r requirements.txt`

In [26]:
import os
from dotenv import load_dotenv

load_dotenv()
assert os.getenv("GOOGLE_API_KEY"), "Defina GOOGLE_API_KEY no ambiente ou em um arquivo .env"
print("GOOGLE_API_KEY carregada.")

GOOGLE_API_KEY carregada.


## 2. Visão geral da arquitetura de um agente

Agentes usam modelos de linguagem para decidir passos, chamar **ferramentas** e manter **memória**. Em LangChain, os blocos principais são:

- **Runtimes/Executores (Runnables/Executors)**: definem o fluxo `prompt → llm → parser` e composições com ferramentas/memória.
- **Ferramentas (Tools)**: funções que o agente pode invocar (cálculo, APIs, banco de dados).
- **Memória (Memory)**: histórico de conversa/estado que influencia respostas futuras.

Fluxo típico:
1. Mensagens de entrada via `PromptTemplate`/`ChatPromptTemplate`.
2. LLM (ex.: Gemini), opcionalmente com ferramentas.
3. Parser (ex.: `StrOutputParser`) para normalizar a saída.
4. Memória (opcional) para manter contexto.
5. Executor/Agente que decide passos e uso de ferramentas.

## 3. Introdução ao LangChain e LCEL

LCEL (LangChain Expression Language) compõe pipelines de forma declarativa. Exemplo: `prompt | llm | parser`. A seguir, configuramos o Gemini e executamos um pipeline simples.

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente conciso e direto."),
    ("human", "Responda: {pergunta}")
])

parser = StrOutputParser()
chain = prompt | llm | parser

resposta = chain.invoke({"pergunta": "Qual é a capital de Portugal?"})
print(resposta)

A capital de Portugal é Lisboa.


## 4. Hello, Agent! (LLM + Parser + Prompt)

Aqui integramos um prompt template, o LLM do Gemini e um parser de saída em um pipeline LCEL minimalista.

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

hello_prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um agente amigável."),
    ("human", "Diga olá com entusiasmo para: {nome}")
])
hello_chain = hello_prompt | llm | StrOutputParser()
print(hello_chain.invoke({"nome": "Agente"}))

Olá Agente! É um prazer conhecê-lo! Espero que você esteja tendo um dia fantástico!


## 5. Extensão: Memória de conversa

Usamos `RunnableWithMessageHistory` para manter histórico por sessão, permitindo respostas contextualizadas.

In [29]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

prompt_mem = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente útil e contextual."),
    ("placeholder", "{history}"),
    ("human", "{input}")
])
chain_mem = prompt_mem | llm | StrOutputParser()

store = {}
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_with_history = RunnableWithMessageHistory(
    chain_mem,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

print("Interação 1:")
out1 = chain_with_history.invoke(
    {"input": "Olá!"},
    config={"configurable": {"session_id": "demo"}}
)
print(out1)

print("Interação 2:")
out2 = chain_with_history.invoke(
    {"input": "Qual foi minha saudação?"},
    config={"configurable": {"session_id": "demo"}}
)
print(out2)

Interação 1:
Olá! Como posso te ajudar hoje?

Interação 2:
Sua saudação foi "Olá!".



## 6. Extensão: Ferramentas (Tools) com Gemini

Registramos uma função como ferramenta e criamos um "agente que chama ferramentas". O LLM escolhe quando invocar a ferramenta e com quais argumentos.

In [30]:
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

@tool
def somar(a: int, b: int) -> int:
    """Soma dois números inteiros.
    Args:
        a: primeiro número
        b: segundo número
    Returns:
        Resultado da soma (int)."""
    return a + b

tools = [somar]

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um agente que usa ferramentas quando necessário."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad") 
])

agent = create_tool_calling_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

resultado = agent_executor.invoke({
    "input": "Some 12 e 30."
})
print("Resposta do agente:", resultado)



> Entering new AgentExecutor chain...
{'name': 'somar', 'description': 'Soma dois números inteiros.\n    Args:\n        a: primeiro número\n        b: segundo número\n    Returns:\n        Resultado da soma (int).', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}

Invoking: `somar` with `{'a': 12.0, 'b': 30.0}`


42{'name': 'somar', 'description': 'Soma dois números inteiros.\n    Args:\n        a: primeiro número\n        b: segundo número\n    Returns:\n        Resultado da soma (int).', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}
O resultado da soma de 12 e 30 é 42.

> Finished chain.
Resposta do agente: {'input': 'Some 12 e 30.', 'output': 'O resultado da soma de 12 e 30 é 42.'}


## 7. Próximos passos
- Troque para `gemini-2.5-pro` quando precisar de respostas mais robustas.
- Adicione ferramentas reais (busca web, banco de dados, APIs internas).
- Persista memória (arquivo, Redis, base de dados) em vez de in-memory.
- Crie testes unitários para ferramentas e fluxos.

Bom proveito!